In [ ]:
import torch
from det3d.torchie import Config

import pickle

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
FINDCENTER_GEN_ONNX = True
POOLFORMER_GEN_ONNX = False

In [ ]:
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu");

In [ ]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model.neck.find_centers(x) 
    
centerFinder = CenterFinder(model)
# centerFinder.cuda()
centerFinder.eval();

In [ ]:
import pickle
# with open("/workspace/centerformer/work_dirs/partition/sample_data/x.pkl", 'wb') as handle:
#     pickle.dump(x, handle)

pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)

In [5]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_order, out_mask = centerFinder(x)

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [6]:
import onnx
import numpy as np

if FINDCENTER_GEN_ONNX:
    model_name = "findCenter.onnx"
    # dummy_input=torch.randn(x.shape, requires_grad=True).cuda()
    dummy_input = torch.as_tensor(np.ones([1, 256, 180, 180]), dtype=torch.float32)
    torch.onnx.export(centerFinder, (dummy_input), model_name,
                input_names=['input_tensor'], 
                output_names=['ct_feat', 'center_pos_embedding', 'out_scores', 'out_labels', 'out_orders', 'out_masks'],
                export_params=True, 
                do_constant_folding=True,
                opset_version=17,
                operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                verbose=True,
                )
    
    # !export POLYGRAPHY_AUTOINSTALL_DEPS=1
    !polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx

    onnx.checker.check_model(onnx.load("findCenter_folded.onnx"))
    print("gen findCenter.onnx success!")
else:
    print("pass")

/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:942: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = scores.detach().cpu().numpy()
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:944: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  order = torch.from_numpy(order).to(labels.device)
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:945: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if yo

[I] RUNNING | Command: /root/anaconda3/envs/centerformer/bin/polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx
[I] Loading model: /data/centerformer/work_dirs/partition/onnx/findCenter.onnx
[I] Original Model:
    Name: torch-jit-export | ONNX Opset: 11
    
    ---- 1 Graph Input(s) ----
    {input_tensor [dtype=float32, shape=(1, 256, 180, 180)]}
    
    ---- 6 Graph Output(s) ----
    {ct_feat [dtype=float32, shape=(1, 3000, 256)],
     center_pos_embedding [dtype=float32, shape=(1, 3000, 256)],
     out_scores [dtype=float32, shape=(6, 1, 500)],
     out_labels [dtype=float32, shape=(6, 1, 500)],
     out_order [dtype=float32, shape=(6, 1, 500)],
     out_mask [dtype=float32, shape=(6, 1, 500)]}
    
    ---- 132 Initializer(s) ----
    
    ---- 641 Node(s) ----
    
[I] Folding Constants | Pass 1
2023-12-13 07:02:01.796384892 [W:onnxruntime:, unsqueeze_elimination.cc:20 Apply] UnsqueezeElimination cannot remove node Unsqueeze_598
2023-1